In [1]:
from flask import Flask, render_template, url_for
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///py.db'
db = SQLAlchemy(app)

class blogInfo(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)

def __repr__(self):
    return '<Task %r>' % self.id

@app.route('/')

def home_page():
    return render_template('index.html')

#@app.route('/admin_post')

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


c:\Users\lori2\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2023 16:12:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:12:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:12:40] "GET / HTTP/1.1" 200 -
